# Processing data after running the Genetic Algorithm
First, we will look at what values the genetic algorithm gave us.   
Then, we will backcalculate the fold changes for each set of model parameters

In [8]:
import tellurium as te
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

# pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.reset_option('^display.', silent=True)

# Viewing the data using pandas

In [10]:
# open the results file
f = open("data/paramData.list", "r")
lines = f.read()
# parse the data into an array 
list_of_lines = lines.splitlines()
f.close()
# reshape into numpy
arr = np.array(list_of_lines)
arr = arr.astype('float64')
arr = np.reshape(arr, (100, 5))
df = pd.DataFrame(arr, columns = ['k1','k2', 'k3','k4','k5'])

g = open("data/scores.list", "r")
gines = g.read()
gist_of_gines = gines.splitlines()
g.close()
garr = np.array(gist_of_gines)
garr = garr.astype('float64')
garr = np.reshape(garr, (100, 1))
gf = pd.DataFrame(garr, columns = ['chiSq'])

data = pd.concat((df, gf),axis=1)
data

,k1,k2,k3,k4,k5,chiSq
0,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
1,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
2,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
3,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
4,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
...,...,...,...,...,...,...
95,261.394702,695.715332,577.783530,884.684705,1102.914821,0.009244
96,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
97,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
98,596.921101,18.871704,13.676371,884.684705,1102.914821,3.050242


In [11]:
# to view data from least to greatest chi square values
data.sort_values(by='chiSq')

,k1,k2,k3,k4,k5,chiSq
0,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
97,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
96,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
79,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
69,224.995922,452.471195,663.815184,884.684705,1102.914821,0.000006
...,...,...,...,...,...,...
43,596.921101,18.871704,13.676371,1321.833009,1102.914821,3.054224
19,146.547502,5.945939,14.625549,884.684705,2198.138011,3.578952
52,224.995922,3.101858,216.036469,346.288361,1163.968418,5.233720
88,224.995922,3.101858,216.036469,346.288361,1275.283509,5.234783


# Backcalculating the predicted fold changes for each set of model parameters

In [6]:
import backCheck
from core import evaluate
from core import models

In [ ]:
backCheck.getFoldChangeValues(paramData="data/paramData.list")

In [7]:
# the original, expected values to compare with the values in 'allData.list'
evaluate.runExperiment(models.groundTruth_mod_e)

array([ 0.69887834,  0.69887834,  0.69887834,  0.69887834, -0.34168351,
        0.0736096 ,  0.0736096 ,  0.0736096 , -0.07181795, -0.37015035,
        0.01547189,  0.01547189, -0.01774854, -0.0914761 , -0.39242256,
        0.0038236 , -0.00471971, -0.02432542, -0.10435341, -0.49949161,
        0.69887834,  0.0736096 ,  0.01547189,  0.0038236 ,  0.00101678])